In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import pandas as pd
import re
import numpy as np


In [3]:
path = './result'
file_list = os.listdir(path)

In [ ]:
tmp = pd.DataFrame()
df_eval = pd.DataFrame()
acc_list = []
def sc_calc_acc_condition_with_temp_with_sc(llm_model, few_shot_n, test_n, q_src_yn, ver, p_ver, sc_num, temp, excel_ver):
    # ./result/sc_l_result_4_15_Y_30_sys_prompt8_0.01_ver1_0.csv
    file_list = os.listdir(path)
    print(file_list)
    opt_file = [x for x in file_list if x.startswith(f'sc_{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}_{ver}_{p_ver}_{sc_num}_{temp}_{excel_ver}')]
    print(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn}_{ver}_{p_ver}_{sc_num}_{temp}_{excel_ver}')
    print(opt_file)
    opt_file = [x for x in opt_file if x.endswith(f'.csv')]
    print(opt_file)

    df = pd.DataFrame()


    # pattern = re.compile(r'.*_[0-9]\.csv$')
    # opt_file = [x for x in opt_file if pattern.match(x)]
    
    
    if len(opt_file)>0 : 
        for f in opt_file:
            print(f)
            tmp = pd.read_csv(f'{path}/{f}', index_col =0)
            tmp = tmp.dropna()

            tmp['gold'] = tmp['answer_encode'].apply(lambda x : re.sub(r'[^012]', '', x))
            tmp['o_result'] = tmp['result'].apply(lambda x : re.sub(r'[^012]', '', x))
            tmp = tmp[tmp['o_result'].isin(['1', '0', '2'])]

            
            gold_df = tmp[['id', 'gold']].drop_duplicates()
            chk_cnt = tmp.groupby(['id', 'o_result']).count().reset_index()[['id', 'o_result', 'question']]
            chk_cnt = chk_cnt.rename(columns = {'question': 'cnt'})
            chk_cnt = chk_cnt[chk_cnt['cnt'] == sc_num]
            chk_cnt = chk_cnt.sort_values(by = ['id', 'cnt'], ascending=[True, False]).groupby(['id']).head(1)
            df_eval = pd.merge(gold_df, chk_cnt, on = ['id'])

            df_eval['equal_yn'] = np.where(df_eval['gold']==df_eval['o_result'], 1, 0)
            acc = (df_eval['equal_yn'].sum()/df_eval.shape[0])*100  
            acc_list.append(acc)
            df = pd.concat([df, df_eval], axis =0)
            
        df['equal_yn'] = np.where(df['gold']==df['o_result'], 1, 0)
        acc = (df['equal_yn'].sum()/df.shape[0])*100            
        print(f'{llm_model}_result_{few_shot_n}_{test_n}_{q_src_yn} : ', acc)


In [5]:
# process3 = Process(target=task, args=('v', 4, 10, 'Y', 10, 'sys_prompt10', 5, 0.01, 'ver5'))
sc_calc_acc_condition_with_temp_with_sc('v', 4, 10, 'Y', 10, 'sys_prompt10', 5,  0.01, 'ver5')

['v_result_4_10_Y_10_sys_prompt10_5_0.01_ver5_0.csv']
v_result_4_10_Y_10_sys_prompt10_5_0.01_ver5
['v_result_4_10_Y_10_sys_prompt10_5_0.01_ver5_0.csv']
['v_result_4_10_Y_10_sys_prompt10_5_0.01_ver5_0.csv']
v_result_4_10_Y_10_sys_prompt10_5_0.01_ver5_0.csv
v_result_4_10_Y :  0.0
